In [1]:
import pandas as pd
from preprocess import Audio_Processor
from sklearn import metrics
from classification_plots import plot_confusion_matrix, plot_learning_curve
import matplotlib.pyplot as plt
import os
from sklearn.mixture import GaussianMixture as GMM

Using TensorFlow backend.


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [2]:
# Able to specify which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"]="1"

SR = 44100
blocksize = 220500
overlap = 0

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Setup

In [4]:
audio_dir = '../ESC-50/audio/'

In [5]:
path_to_db='../ESC-50/'
ps = Audio_Processor(path_to_db + 'audio/')

# Load Dataset
Here we load the csv that describes each file in the dataset. We add a high level category that is defined in the ESC-50 documentation. This we realize is anthetical to true training, it is a stopgap for when we use NLP to classify tags into these categories.

In [6]:
dataset = pd.read_csv(path_to_db + 'meta/esc50.csv')
classes = [None] * 50

In [7]:
h_classes = ['Human & Animal', 'Interacting Materials']
mapping = {'dog': 0,'rooster': 0,'pig': 0,'cow': 0,'frog': 0,'cat': 0,'hen': 0,
            'insects': 0,'sheep': 0,'crow': 0,'rain': 1,'sea_waves': 1,'crackling_fire': 1,
            'crickets': 0,'chirping_birds': 0,'water_drops': 1,'wind': 1,'pouring_water': 1,
            'toilet_flush': 1,'thunderstorm': 1,'crying_baby': 0,'sneezing': 0,'clapping': 0,
            'breathing': 0,'coughing': 0,'footsteps': 1,'laughing': 0,'brushing_teeth': 1,
            'snoring': 0,'drinking_sipping': 1,'door_wood_knock': 1,'mouse_click': 1,
            'keyboard_typing': 1,'door_wood_creaks': 1,'can_opening': 1,'washing_machine': 1,
            'vacuum_cleaner': 1,'clock_alarm': 1,'clock_tick': 1,'glass_breaking':1,'helicopter': 1,
            'chainsaw': 1,'siren': 1,'car_horn': 1,'engine': 1,'train': 1,'church_bells': 1,
            'airplane': 1,'fireworks': 1,'hand_saw': 1,
            }

In [8]:
dataset['h_category'] = None
for index, row in dataset.iterrows():
    target = row['target']
    classes[target] = row['category']
    dataset.loc[index, 'h_category'] = mapping[row['category']]

In [9]:
dataset.head()

,filename,fold,target,category,esc10,src_file,take,h_category
0,1-100032-A-0.wav,1,0,dog,True,100032,A,0
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,0
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,1
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,1
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,1


## Getting Preprocessed Data
We allow for previously preprocessed data to be retrieved for faster training turnaround. If the fold has been preprocessed, it is loaded but if not it is processed and saved.

In [10]:
df = ps.preprocess_fold(dataset, 
                        kind='mfcc', 
                        fld=range(2,6), 
                        blocksize=blocksize, 
                        overlap=overlap)

In [11]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,l_target,h_target
count,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,...,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1.600000e+03,1600.000000,1600.000000
mean,51.308986,55.436622,37.072319,32.665469,29.089603,27.197671,25.194174,25.108523,23.179477,22.828157,...,-0.003767,0.002611,-0.002889,0.002254,-0.002463,0.001378,-0.001494,7.538308e-04,0.620000,24.500000
std,23.801254,28.183104,15.718001,13.332764,11.160740,10.515712,9.637630,9.675467,8.789096,8.545873,...,0.011821,0.011696,0.010898,0.010615,0.010112,0.009818,0.009729,9.453870e-03,0.485538,14.435381
min,1.747074,1.479881,1.615058,1.539040,1.437081,1.393894,1.404186,1.518345,1.486071,1.272155,...,-0.075351,-0.043938,-0.075981,-0.041356,-0.070834,-0.038708,-0.071320,-4.075538e-02,0.000000,0.000000
25%,33.332886,32.550216,25.569366,22.457447,20.671324,19.468957,18.142160,18.336386,16.718161,16.513697,...,-0.009462,-0.003002,-0.008140,-0.003326,-0.007607,-0.003619,-0.006616,-4.387224e-03,0.000000,12.000000
50%,49.849501,52.401495,36.549704,32.329287,29.294185,27.021204,24.971081,24.817577,23.047764,22.483568,...,-0.001737,0.000776,-0.001195,0.000719,-0.001023,0.000192,-0.000428,9.772273e-10,1.000000,24.500000
75%,67.509731,75.187595,48.118331,41.837019,36.495502,34.406650,31.754910,31.376616,29.460840,28.974748,...,0.002094,0.007613,0.002531,0.006822,0.002604,0.005863,0.003657,5.833929e-03,1.000000,37.000000
max,138.338211,143.565491,93.059975,82.682236,65.922852,69.506638,54.715382,79.771935,55.127274,53.718040,...,0.033273,0.075946,0.033666,0.073340,0.041846,0.061704,0.062855,7.767285e-02,1.000000,49.000000


In [12]:
from sklearn.model_selection import train_test_split
train, test, = train_test_split(
                        df, test_size=0.20, random_state=42)

In [13]:
train_X = train.drop(['l_target', 'h_target'], axis=1)
train_y = train['l_target']
test_X = test.drop(['l_target', 'h_target'], axis=1)
test_y = test['l_target']

# Shallow Net
Separate into classes

In [14]:
from sklearn.model_selection import GridSearchCV
params = {
          'n_components': range(2,20,2),
          'covariance_type': ('full', 'tied', 'diag', 'spherical'),
          'init_params': ('kmeans', 'random')
         }

gmm = GMM(max_iter=300)

clf = GridSearchCV(gmm, params, cv=5)

clf.fit(train_X, train_y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=300,
        means_init=None, n_components=1, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'covariance_type': ('full', 'tied', 'diag', 'spherical'), 'n_components': range(2, 20, 2), 'init_params': ('kmeans', 'random')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

0.14375
Confusion matrix, without normalization
[[24  9 84  9]
 [58 22 71 43]
 [ 0  0  0  0]
 [ 0  0  0  0]]


In [15]:
pred = clf.predict(test_X)
print(metrics.accuracy_score(test_y, pred))
cm = metrics.confusion_matrix(test_y, pred)
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, h_classes)
plt.show()

# All Data

In [16]:
all_df = ps.preprocess_fold(dataset, 
                        kind='mfcc', 
                        fld=1, 
                        blocksize=blocksize, 
                        overlap=overlap)

In [17]:
X = all_df.drop(['l_target', 'h_target'], axis=1)
y = all_df['l_target']

In [ ]:
pred = clf.predict(X)
print(metrics.accuracy_score(y, pred))
cm = metrics.confusion_matrix(y, pred)
plt.figure(figsize=(20,20))
plot_confusion_matrix(cm, h_classes)
plt.show()